In [1]:
import kagglehub
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

**Pull in Data**

In [5]:
# Download latest version
path = kagglehub.dataset_download("nikhil7280/student-performance-multiple-linear-regression")
print("Path to dataset files:", path)

student_data_df : pd.DataFrame = pd.read_csv(path+"/Student_Performance.csv")

Path to dataset files: /root/.cache/kagglehub/datasets/nikhil7280/student-performance-multiple-linear-regression/versions/1


In [6]:
print("preprocessing...")
clean_student_data_df = student_data_df

null_count = student_data_df.isnull().sum().sum()
if null_count:
    clean_student_data_df = clean_student_data_df.dropna()

preprocessing...


In [7]:
dup_count = student_data_df.duplicated().sum()
if dup_count:
    clean_student_data_df = clean_student_data_df.drop_duplicates()

In [8]:
encoder = LabelEncoder()

categorical_columns = clean_student_data_df.select_dtypes(include=['object']).columns

for ccol in categorical_columns:
    clean_student_data_df[ccol+"_encoded"] = encoder.fit_transform(clean_student_data_df[ccol])
    clean_student_data_df.drop(columns=ccol, inplace=True)

<ipython-input-8-a2d24d432f1f>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_student_data_df[ccol+"_encoded"] = encoder.fit_transform(clean_student_data_df[ccol])
<ipython-input-8-a2d24d432f1f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_student_data_df.drop(columns=ccol, inplace=True)


In [11]:
print("build model...")
target_variable = 'Performance Index'

X = clean_student_data_df.drop(target_variable, axis=1)
y = clean_student_data_df[target_variable]

build model...


In [12]:
print("model selection...")

# Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.1),
    "Random Forest": RandomForestRegressor(n_estimators=100),
    "Support Vector Regression": SVR(kernel='linear'),
    "K-Nearest Neighbors": KNeighborsRegressor(n_neighbors=5)
}

# Evaluate models using cross-validation (e.g., 5-fold)
cv_results = {}
for model_name, model in models.items():
    # Compute cross-validation scores (negative MSE)
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_log_error')

    # Store the mean of the cross-validation scores (lower is better)
    mse = -scores.mean()
    cv_results[model_name] = mse

# Display cross-validation results
lowest_mse = 1000
for model_name, mse in cv_results.items():
    print(f"{model_name}: CV MSE = {mse}")

    if mse < lowest_mse:
        lowest_mse = mse
        best_model = model_name

print(f'best model: {best_model}')
model = models[best_model]

model selection...
Linear Regression: CV MSE = 0.046482825947030154
Ridge Regression: CV MSE = 0.04648291149616977
Lasso Regression: CV MSE = 0.04675971652116804
Random Forest: CV MSE = 0.05367200020238731
Support Vector Regression: CV MSE = 0.0464786164844271
K-Nearest Neighbors: CV MSE = 0.05659055703751305
best model: Support Vector Regression


In [14]:
print("evaluation...")

scores = cross_val_score(model, X, y, cv=5, scoring='neg_root_mean_squared_log_error')
final_mse = -scores.mean()

print("MSE:", final_mse)


evaluation...
MSE: 0.0464786164844271
